# P. Visión

In [9]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm

In [10]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [11]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [27]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroAleatoriasPorFoto = 3       # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]

    rowsFotoEntorno = dataNeg[0].shape[0]
    colsFotoEntorno = dataNeg[0].shape[1]

    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        for j in range(numeroAleatoriasPorFoto):
            randRow = random.randint(0,rowsFotoEntorno - rowsFotoPersona)
            randCol = random.randint(0,colsFotoEntorno - colsFotoPersona)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroAleatoriasPorFoto*i] = randomImage 
    return fotosNegTrain

In [28]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0]))
    
    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i] = hog(imagenes[i])
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [39]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

In [40]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [41]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

In [44]:
clf = svm.SVC()
clf.fit(fotosTrain,y)

C:\Users\Jhonny Chicaiza\Anaconda3\envs\py37vision\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonny Chicaiza\Anaconda3\envs\py37vision\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [48]:
clf.score(fdPosTrain,yPos)

0.9218472468916519

In [63]:
def secImagen(imagen,maskHeight,maskWidth,desp):
    arrayImagen = {}
    aux = 0
    for i in range(0,imagen.shape[0],desp):        
        for j in range(0,imagen.shape[1],desp):
            if (i+maskHeight < imagen.shape[0] and j+maskWidth < imagen.shape[1]):
                arrayImagen[aux] = imagen[i:i+maskHeight,j:j+maskWidth]
                aux = aux + 1
    return arrayImagen

In [64]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotostest = leerFotos(path)

In [65]:
partImagen = secImagen(fotostest[6],134,70,10)

In [66]:
cv2.imshow('xd ', secImagen)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: mat is not a numpy array, neither a scalar